- 기계번역
- 챗봇

요즘에는 많이 사용하지 않음 <- Transformer가 나와서
RNN은 시계열 쪽에서 사용

# Module

- 기계번역
- 챗봇

요즘에는 많이 사용하지 않음 <- Transformer가 나와서

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings(action='ignore')

2023-08-03 10:32:44.978247: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 10:32:45.038327: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-03 10:32:45.039979: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 10:32:46.063467: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
df.shape

(11823, 3)

In [4]:
df.isnull().sum()

Q        0
A        0
label    0
dtype: int64

# Data preprocessing

In [5]:
texts = []
pairs = []

for i in range(len(df)):
  texts.append(df.iloc[i,0])
  pairs.append(df.iloc[i,1])

In [6]:
# 특수 문자 제거
def clean_sentence(sentence):
    # 한글, 숫자를 제외한 모든 문자는 제거합니다.
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence

In [1]:
!pip install konlpy # 코랩에서는 항상 설치 해줘야함

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.4.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (465 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 10.3 MB/s eta 0:00:0000:0100:01


In [7]:
from konlpy.tag import Okt
okt = Okt()

In [8]:
# 형태소 반환
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence)) # morphs: 형태소 분석 -> 리스트 반환

In [9]:
texts[1]

'1지망 학교 떨어졌어'

In [10]:
okt.morphs(texts[1])

['1', '지망', '학교', '떨어졌어']

In [12]:
' '.join(okt.morphs(texts[1]))

'1 지망 학교 떨어졌어'

In [13]:
def clean_and_morph(sentence, is_question=True):
    # 한글 문장 전처리
    sentence = clean_sentence(sentence)
    # 형태소 변환
    sentence = process_morph(sentence)
    # Question 인 경우, Answer인 경우를 분기하여 처리합니다.
    if is_question:
        return sentence
    else:
        # START 토큰은 decoder input에 END 토큰은 decoder output에 추가합니다.
        return ('<START> ' + sentence, sentence + ' <END>') # <START> : <SOS>, <END> : <EOS>

In [14]:
s = pairs[1]
s

'위로해 드립니다.'

In [17]:
'<START> ' + s, s + ' <END>'

('<START> 위로해 드립니다.', '위로해 드립니다. <END>')

In [18]:
def preprocess(texts, pairs):
    questions = [] # encoder input
    answer_in = [] # decoder input
    answer_out = [] # decoder output

    # 질의에 대한 전처리
    for text in texts:
        # 전처리와 morph 수행
        question = clean_and_morph(text, is_question=True)
        questions.append(question)

    # 답변에 대한 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)

    return questions, answer_in, answer_out

In [19]:
questions, answer_in, answer_out = preprocess(texts, pairs)

In [20]:
print(questions[:5]) # encoder_input

['12시 땡', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', '심하네']


In [21]:
print(answer_in[:5]) # decoder input

['<START> 하루 가 또 가네요', '<START> 위로 해 드립니다', '<START> 여행 은 언제나 좋죠', '<START> 여행 은 언제나 좋죠', '<START> 눈살 이 찌푸려지죠']


In [22]:
print(answer_out[:5]) # decoder output

['하루 가 또 가네요 <END>', '위로 해 드립니다 <END>', '여행 은 언제나 좋죠 <END>', '여행 은 언제나 좋죠 <END>', '눈살 이 찌푸려지죠 <END>']


In [23]:
all_sentences = questions + answer_in + answer_out

In [26]:
# Tokenizer
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>') 
# filters='' : 특수문자를 제거 안함
# lower=False : 대문자를 소문자로 변환 안함
# oov_token : out of vocabulary

In [27]:
tokenizer.fit_on_texts(all_sentences)

In [28]:
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t\t => \t{idx}')
    if idx > 10:
        break

<OOV>		 => 	1
<START>		 => 	2
<END>		 => 	3
이		 => 	4
을		 => 	5
거		 => 	6
가		 => 	7
예요		 => 	8
사람		 => 	9
요		 => 	10
에		 => 	11


In [29]:
len(tokenizer.word_index)

12637

In [30]:
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

In [31]:
question_padded = pad_sequences(question_sequence, maxlen=30, truncating='post', padding='post')
answer_in_padded = pad_sequences(answer_in_sequence, maxlen=30, truncating='post', padding='post')
answer_out_padded = pad_sequences(answer_out_sequence, maxlen=30, truncating='post', padding='post')

In [32]:
question_padded.shape

(11823, 30)

In [33]:
print('인코더의 입력의 크기(shape) :',question_padded.shape)
print('디코더의 입력의 크기(shape) :',answer_in_padded.shape)
print('디코더의 레이블의 크기(shape) :',answer_out_padded.shape)

인코더의 입력의 크기(shape) : (11823, 30)
디코더의 입력의 크기(shape) : (11823, 30)
디코더의 레이블의 크기(shape) : (11823, 30)


In [34]:
vocab_size = len(tokenizer.word_index) + 1 # +1 : 0번 패딩 토큰을 고려하여 +1
print("단어 집합의 크기 : ",vocab_size)

단어 집합의 크기 :  12638


In [35]:
word_to_index = tokenizer.word_index
index_to_word = tokenizer.index_word

In [36]:
index_to_word[1]

'<OOV>'

# 모형

In [46]:
VOCAB_SIZE = len(tokenizer.word_index)+1

## Encoder

In [40]:
# # 실습용 코드
# class Encoder(tf.keras.Model): # tf.keras.Model / tf.keras.layers.Layer

#     def __init__(self, vocab_size, embedding_dim, hidden_unit, time_steps):
#         super().__init__()
#         self.vocab_size = vocab_size
#         self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=time_steps)
#         self.dropout = tf.keras.layers.Dropout(0.5)
#         self.lstm = tf.keras.layers.LSTM(hidden_unit, retrurn_sequences=True, return_state=True)

#     def call(self, x, training=None, mask=None):
#         x = self.embedding(x)
#         x = self.dropout(x)
#         x, h, c = self.lstm(x)
#         return h, c

In [ ]:
# enc = Encoder(vocab_size, 30, 20, 30)
# enc(question_padded[0].reshape(1, -1))

In [42]:
class Encoder(tf.keras.Model):

  # hidden_unit : 은닉층의 노드 수, vocab_size : 단어 사전의 크기, embedding_dim : 임베딩 벡터의 차원 수, time_steps : 입력 시퀀스의 길이
  def __init__(self, units, vocab_size, embedding_dim, time_steps): 
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=time_steps)
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.lstm = tf.keras.layers.LSTM(units, return_state=True, return_sequences=True)


  def call(self,inputs):

    x = self.embedding(inputs)
    x = self.dropout(x)
    x, hidden_state, cell_state = self.lstm(x)

    return hidden_state, cell_state

Tensorflow에서는 return state = True로 해주어야 hidden state와 cell state가 출력이 됩니다. return sequences = True를 해주게 되면 각 time step마다의 출력값을 내보내주게 됩니다.

In [44]:
question_padded[0]

array([5962, 8808,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [43]:
sample = question_padded[0].reshape(1,-1) # batch size를 고려해서 1로 reshape
sample.shape

(1, 30)

In [47]:
emb = tf.keras.layers.Embedding(VOCAB_SIZE,100,input_length=30)(sample)
emb.shape

TensorShape([1, 30, 100])

In [48]:
drop = tf.keras.layers.Dropout(0.2)(emb)

In [49]:
# 아무런 인자를 안 주었을 경우
x = tf.keras.layers.LSTM(128)(drop)
x.shape

TensorShape([1, 128])

가장 마지막 time step의 출력값만을 보여준

In [50]:
# return sequence = True
x = tf.keras.layers.LSTM(128,return_sequences=True)(drop)
x.shape

TensorShape([1, 30, 128])

각각의 time step마다의 출력값을 보여준다

In [51]:
# return state = True, return sequences = True
x,h,c = tf.keras.layers.LSTM(128,return_sequences=True,return_state=True)(drop)
x.shape,h.shape, c.shape

(TensorShape([1, 30, 128]), TensorShape([1, 128]), TensorShape([1, 128]))

가장 마지막 층의 은닉 상태와 cell state도 같이 출력을 해주게 된다

## Decoder

In [52]:
class Decoder(tf.keras.Model):

  def __init__(self, units, vocab_size, embedding_dim, time_steps):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=time_steps)
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.lstm = tf.keras.layers.LSTM(units,return_sequences=True,return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size, activation='softmax')

  def call(self, x, hidden, cell):
    x = self.embedding(x)
    x = self.dropout(x)
    # Encoder의 context vector 반영
    x,h,c = self.lstm(x,initial_state=[hidden,cell])

    x = self.dense(x)
    return x,h,c

In [53]:
sample2 = answer_in_padded[0].reshape(1,-1)
sample2.shape

(1, 30)

In [54]:
emb2 = tf.keras.layers.Embedding(VOCAB_SIZE,100,input_length=30)(sample2)
emb2.shape

TensorShape([1, 30, 100])

In [55]:
x2,h2,c2 = tf.keras.layers.LSTM(128,return_sequences=True,return_state=True)(emb2,initial_state=[h,c])
x2.shape,h2.shape,c2.shape

(TensorShape([1, 30, 128]), TensorShape([1, 128]), TensorShape([1, 128]))

## Seq2Seq

In [56]:
class Seq2Seq(tf.keras.Model):

  def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
    super().__init__()
    self.start = start_token # SOS token
    self.end = end_token # EOS token
    self.time_steps = time_steps

    self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
    self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

  def call(self, x, training=False, mask=None):


    # 학습
    if training:
      enc_input, dec_input = x
      hidden, cell = self.encoder(enc_input)
      outputs,_,_ = self.decoder(dec_input, hidden, cell)
      return outputs

    # 추론
    else:
      hidden, cell = self.encoder(x)
      # SOS token을 가장 먼저 넣어줌
      target_seq = tf.constant([[self.start]],dtype=tf.float32) # constant : 상수 취급(학습을 안하겠다)
      # 결과값을 담는 tensor
      results = tf.TensorArray(tf.int32,self.time_steps) # TensorArray : 텐서를 담는 배열(아직 메모리상에 올라가지 않음) -> 메모리를 효율적으로 사용 가능

      for i in tf.range(self.time_steps):
        output, decoder_hidden, decoder_cell = self.decoder(target_seq, hidden, cell)
        output = tf.cast(tf.argmax(output, axis=-1), dtype=tf.int32) # cast = 넘파이의 astype : 자료형 변환 / argmax : 최대값의 인덱스를 반환 -> 굳이 정수형으로 바꿔줘야하나?
        output = tf.reshape(output, shape=(1,1))
        results = results.write(i,output)

        # EOS token을 만나면 문장 생성 중지
        if output == self.end:
          break

        # 이전 값들 최신화
        target_seq = output
        hidden = decoder_hidden
        cell = decoder_cell
      return tf.reshape(results.stack(), shape=(1,self.time_steps))


### 학습 시

In [201]:
encoder = Encoder(128,VOCAB_SIZE,100,30)
decoder = Decoder(128,VOCAB_SIZE,100,30)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 72, in error_handler
    del filtered_tb  File "/usr/local/lib/python3.10/dist-packages/keras/engine/base_layer.py", line 1105, in __call__
    with call_context.enter(  File "/usr/local/lib/python3.10/dist-packages/keras/utils/traceback_utils.py", line 158, in error_handler
    del bound_signature  File "<ipython-input-179-cf15b3f73bb2>", line 44, in call
    return tf.reshape(results.stack(), shape=(1,self.time_steps))  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


In [202]:
hidden, cell = encoder(sample)
hidden.shape, cell.shape

(TensorShape([1, 128]), TensorShape([1, 128]))

In [203]:
output, _, _ = decoder(sample2, hidden, cell)
output.shape

TensorShape([1, 30, 12640])

### 추론 시

In [204]:
hidden, cell = encoder(sample)
hidden.shape, cell.shape

(TensorShape([1, 128]), TensorShape([1, 128]))

In [205]:
start = tokenizer.word_index['<START>']
start

2

In [206]:
target_seq = tf.constant([[start]],dtype=tf.float32)
print(target_seq.shape)
print(target_seq)

(1, 1)
tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


In [207]:
# 동적 크기 배정
results = tf.TensorArray(tf.int32,30)
results.read(0)

ValueError: ignored

tf.TensorArray는 동적인 객체인데 현재 아무 값도 넣어주지 않았기 때문에 오류가 발생하게 된다

In [208]:
output, decoder_hidden, decoder_cell = decoder(target_seq,hidden,cell)
# 가장 확률이 높은 단어를 고르고 tf.cast를 통해 형 변환
output = tf.cast(tf.argmax(output,axis=-1),dtype=tf.int32)
output

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[9150]], dtype=int32)>

In [209]:
output = tf.reshape(output,shape=(1,1))
output

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[9150]], dtype=int32)>

In [210]:
results = results.write(0,output)

In [211]:
results.read(0)

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[9150]], dtype=int32)>

In [212]:
results = tf.TensorArray(tf.int32,30)
target_seq = tf.constant([[start]],dtype=tf.float32)

In [214]:
for i in tf.range(30):
        output,decoder_hidden, decoder_cell = decoder(target_seq,hidden,cell)
        output = tf.cast(tf.argmax(output,axis=-1),dtype=tf.int32)
        output = tf.reshape(output,shape=(1,1))
        results = results.write(i,output)

        # EOS token을 만나면 문장 생성 중지
        if output == tokenizer.word_index['<END>']:
          break

        # 이전 값들 최신화
        target_seq = output
        hidden = decoder_hidden
        cell = decoder_cell

In [215]:
results.stack()

<tf.Tensor: shape=(30, 1, 1), dtype=int32, numpy=
array([[[11278]],

       [[ 2015]],

       [[ 6711]],

       [[12080]],

       [[12080]],

       [[ 9158]],

       [[12277]],

       [[ 8070]],

       [[10872]],

       [[ 9181]],

       [[ 9181]],

       [[12037]],

       [[ 9181]],

       [[12037]],

       [[ 9181]],

       [[ 8140]],

       [[ 1957]],

       [[12075]],

       [[ 2752]],

       [[ 1865]],

       [[ 2144]],

       [[11312]],

       [[ 5107]],

       [[ 5920]],

       [[ 1622]],

       [[  988]],

       [[ 7794]],

       [[ 7794]],

       [[  959]],

       [[ 5765]]], dtype=int32)>

In [216]:
tf.reshape(results.stack(),shape=(1,30))

<tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[11278,  2015,  6711, 12080, 12080,  9158, 12277,  8070, 10872,
         9181,  9181, 12037,  9181, 12037,  9181,  8140,  1957, 12075,
         2752,  1865,  2144, 11312,  5107,  5920,  1622,   988,  7794,
         7794,   959,  5765]], dtype=int32)>

# 학습

In [217]:
EMBEDDING_DIM = 100
TIME_STEPS = 30
START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']

UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index)+1
DATA_LENGTH = len(questions)

In [218]:
model = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)

In [219]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc']) # one-hot encoding을 하지 않았기 때문에 sparse_categorical_crossentropy를 사용

In [220]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3,mode='min',verbose=1)
checkpoint_path = 'seq2seq_training_checkpoint.h5'
mc = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='loss',
                             verbose=1
                            )

In [221]:
model.fit([question_padded, answer_in_padded],answer_out_padded,epochs=350,batch_size=64, callbacks=[mc,es])

Epoch 1/350
185/185 [==============================] - ETA: 0s - loss: 2.5483 - acc: 0.7842
Epoch 1: loss improved from inf to 2.54830, saving model to seq2seq_training_checkpoint.h5
185/185 [==============================] - 19s 82ms/step - loss: 2.5483 - acc: 0.7842
Epoch 2/350
185/185 [==============================] - ETA: 0s - loss: 1.3132 - acc: 0.8161
Epoch 2: loss improved from 2.54830 to 1.31325, saving model to seq2seq_training_checkpoint.h5
185/185 [==============================] - 6s 33ms/step - loss: 1.3132 - acc: 0.8161
Epoch 3/350
185/185 [==============================] - ETA: 0s - loss: 1.2301 - acc: 0.8239
Epoch 3: loss improved from 1.31325 to 1.23014, saving model to seq2seq_training_checkpoint.h5
185/185 [==============================] - 5s 26ms/step - loss: 1.2301 - acc: 0.8239
Epoch 4/350
184/185 [============================>.] - ETA: 0s - loss: 1.1768 - acc: 0.8285
Epoch 4: loss improved from 1.23014 to 1.17708, saving model to seq2seq_training_checkpoint.h5


# 예측

In [222]:
def convert_index_to_text(indexs, end_token):

    sentence = ''

    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == end_token:
            # 끝 단어이므로 예측 중비
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''

        # 빈칸 추가
        sentence += ' '
    return sentence

In [223]:
def make_prediction(model, question_inputs):
    results = model(x=question_inputs, training=False)  # 추론이기 때문에 training=False
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

In [224]:
# 자연어 (질문 입력) 대한 전처리 함수
def make_question(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=30, truncating='post', padding='post')
    return question_padded

In [225]:
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(model, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

In [226]:
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))

<< 말을 걸어 보세요!
박규찬이 누구야
>> 챗봇 응답: 그 는 신 이야 
<< 말을 걸어 보세요!
박규찬 알아?
>> 챗봇 응답: 그 는 신 이야 
<< 말을 걸어 보세요!
저녁에 뭐 먹을까
>> 챗봇 응답: 냉장고 파먹기 해보세요 
<< 말을 걸어 보세요!
오늘 피곤해
>> 챗봇 응답: 속 쓰리겠어요 
<< 말을 걸어 보세요!
q


In [ ]:
model.load_weights('/content/seq2seq_training_checkpoint.h5')